## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [21]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")

train = df[:210]
test = df[210:]

train.shape, test.shape
train.head()

,age,length,diameter,height,weight,gender
0,6,0.474627,0.211352,0.178189,78.971766,1
1,1,0.465847,0.339388,0.170522,98.781960,1
2,4,0.122807,0.238691,0.106924,88.792625,0
3,4,0.204579,0.360543,0.034261,1.028847,0
4,8,0.243458,0.358037,0.128080,6.503367,0


In [28]:
from statsmodels.formula.api import logit
model = logit('gender ~ weight', data=train).fit()

import numpy as np
round(np.exp(model.params['weight']), 4)

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4


np.float64(1.0047)

### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [23]:
model2 = logit('gender ~ age + length + diameter + height + weight', data=train).fit()
model2.summary()

round(-2 * model2.llf, 2)

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4


np.float64(286.93)

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [30]:
pred = model.predict(test) > 0.5

from sklearn.metrics import accuracy_score
score = accuracy_score(pred, test['gender'])
print(1-score)

0.47777777777777775


In [26]:
# 다른 풀이 (사이킷런)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습 데이터와 테스트 데이터 분리
X_train = train[['weight']]
y_train = train['gender']
X_test = test[['weight']]
y_test = test['gender']

# 2) 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 3) 테스트 데이터를 사용해 예측
pred = model.predict(X_test)
pred

# 4) 실제 값과 예측값을 사용하여 정확도 계산
# acc = accuracy_score(y_test, pred)

# 5) 오류율 계산
# print(round(1 - acc, 3))

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0])

### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [31]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")

In [32]:
df.corr()
# 0.434

,ERP,Feature1,Feature2,Feature3,CPU
ERP,1.000000,0.434442,0.076116,-0.059888,-0.201027
Feature1,0.434442,1.000000,0.076148,0.049980,-0.098571
Feature2,0.076116,0.076148,1.000000,0.126589,0.071456
Feature3,-0.059888,0.049980,0.126589,1.000000,-0.035247
CPU,-0.201027,-0.098571,0.071456,-0.035247,1.000000


### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [36]:
cond = df['CPU'] < 100
df2 = df[cond]
# df2.head()
from statsmodels.formula.api import ols
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data=df2).fit()
model.summary()
# 0.226


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Tue, 25 Nov 2025   Prob (F-statistic):           6.07e-05
Time:                        11:44:14   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.000      44.470     104.399
Feature1       0.1391      0.030      4.583      0.000       0.079       0.199
Feature2       0.0966      0.129      0.747      0.457      -0.160       0.353
Feature3      -0.4136      0.395     -1.046      0.298      -1.198       0.371
CPU           -0.1125      0.061     -1.846      0.068      -0.233       0.008
==============================================================================
Omnibus:                        0.279   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.870   Jarque-Bera (JB):                0.442
Skew:                           0.090   Prob(JB):                        0.802
Kurtosis:                       2.729   Cond. No.                     1.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
# 0.457




